Usando el archivo `datos.csv` resuelva las siguientes preguntas. Para los cálculos use el lenguage Python. El archivo contiene 10 variables independientes y una variable de salida. No todas las variables independientes son relevantes. La variable de salida indica la clase a la que pertenece cada uno de los ejemplos.

# Pregunta 1

Si se usa un algoritmo de detección de grupos (como k-means), el algoritmo es capaz de detectar los grupos existentes y asignar correctamente los registros a cada grupo? Podría escogerse un subconjunto de variables independientes para mejorar la detección de grupos? argumente.

Los algoritmos de detección de grupos o clusters utilizan un aprendizaje no supervisado. Esto quiere decir que no se definen unas clases predefinidas como variable respuesta para él clasificar en dichas clases, sino que es capaz de detectar los grupos o clusters presentes en una muestra y asignar correctamente los registros pertenecientes a cada grupo. Algunas de las variables independientes pueden tener ruido que haga que la detección de grupos sea peor, por lo que se puede escoger un subconjunto de ellas con el fin mejorar la detección. Adicionalmente, es necesario seleccionar las variables más relevantes para garantizar parsimonia. Lo primero, es calcular el modelo:

In [1]:
#Carga de las librerias necesarias
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans


#Selección del directorio de trabajo
os.chdir('/home/agch2/Dropbox/Asignaturas 2017-I MIS/Aprendizaje de maquinas/machine-learning-master')

#Carga del archivo en un dataframe
df = pd.read_table("datos.csv",sep=",")

#Selección de variables independientes y dependiente (la última no es usada para calibrar el modelo sino solo para ver que tal pronostica, ya que es un algoritmo no supervisado)
indep=df.iloc[:,range(10)]
depend=np.array(df.loc[:,"class"], dtype=int)

#Ajuste del modelo:
kmeans = KMeans(n_clusters=3, random_state=0).fit(indep)
kmeans.labels_
kmeans.cluster_centers_
kmeans_predict=kmeans.predict(indep)
kmeans_predict

kmeans_predict[kmeans_predict==2]=[3]
kmeans_predict[kmeans_predict==0]=[4]
kmeans_predict[kmeans_predict==1]=[5]
kmeans_predict=kmeans_predict-2
kmeans_predict

#Rendimiento del modelo:
rend_Km=np.mean(depend==kmeans_predict)
rend_Km

ImportError: No module named 'pandas'

Se puede observar que la predicción del modelo es de un 100%. Ahora procedemos a seleccionar las variables principales. K-means no cuenta con una función para la selección de variables y la función SelectFromModel, la cual es utilizada en otros algoritmos como regresión logística y árboles de desición, no funciona para k-means ya que este clasificador no permite la obtención de sus parámetros. Así, se realiza la selección de variables de la siguiente forma:

- Se aplica el algoritmo añadiendo una por una cada variable independiente. Es decir, se empieza utilizando solamente una de ellas y luego se utiliza otra, extrayendo la variable incluida anteriormente. Esto se hace sucesivamente para cada una de las variables. De todas las variables, finalmente la primera variable que se incluye en el modelo, es quella que mayor aumento genera en su rendimiento.
- El procedimiento anterior se hace repetidamente hasta que el rendimiento se estabilice.

In [35]:
rend_tot_fin=[]
S_F_tot=[]
V_S_F=range(indep.shape[1])
indep2=indep[:]


for j in range(indep.shape[1]):  #Ver como cambia el rendimiento mientras aumento una por una mis variables

    rend_tot=[]
    
    for i in V_S_F:  #Selecionar la variable que genera mayor aumento en el rendimiento de cada corrida
        
        if j==0:
            indep_F_S=indep.iloc[:,[i]]
        else:
            indep_F_S=indep.iloc[:,list(set().union(S_F_tot,[i]))]
            
        #print(list(set().union(seq1,seq2)))
        #print(indep_F_S)

        kmeans = KMeans(n_clusters=3, random_state=0).fit(indep_F_S)
        kmeans.labels_
        kmeans.cluster_centers_
        kmeans_predict=kmeans.predict(indep_F_S)
        #kmeans_predict

        kmeans_predict[kmeans_predict==2]=[3]
        kmeans_predict[kmeans_predict==0]=[4]
        kmeans_predict[kmeans_predict==1]=[5]
        kmeans_predict=kmeans_predict-2
        #kmeans_predict

        rend_Km=np.mean(depend==kmeans_predict)
        #rend_Km
        #print("%.3f" % rend_Km)

        rend_tot.append(rend_Km)

    print(rend_tot)
    m = max(rend_tot)
    S_F=[i for i, j in enumerate(rend_tot) if j == m]
    if j==0:
        S_F=S_F[:]
    else:
        for k in S_F_tot:
            if S_F[0]>=k:
                S_F[0]=S_F[0]+1
            else: S_F[0]=S_F[0]
    S_F_tot.append(S_F[0])
    
    seq1=list(range(indep2.shape[1]))[0:S_F[0]]
    seq2=list(range(indep2.shape[1]))[S_F[0]+1:10]
    V_S_F=list(set().union(seq1,seq2))
    indep2=indep2.iloc[:,V_S_F]
    #print(V_S_F)
    
    rend_tot_fin.append(m)

#print(rend_tot_fin)

m2 = max(rend_tot_fin)
S_F_Max=[i for i, j in enumerate(rend_tot_fin) if j == m2]

S_F_Variables=S_F_tot[0:S_F_Max[0]+1]
print("Variables seleccionadas:   ","x",S_F_Variables[0],",   x",S_F_Variables[1],"\nRendimiento:",m2)

[0.68666666666666665, 0.35999999999999999, 0.32666666666666666, 0.32444444444444442, 0.71333333333333337, 0.65111111111111108, 0.36222222222222222, 0.33111111111111113, 0.3288888888888889, 0.18222222222222223]
[1.0, 0.71777777777777774, 0.61777777777777776, 0.18222222222222223, 0.58444444444444443, 0.62, 0.14888888888888888, 0.71111111111111114, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0]
Variables seleccionadas:    x 4 ,   x 0 
Rendimiento: 1.0


In [36]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,class
0,0.217051,2.301448,2.063373,-2.877469,-0.280745,3.037343,1.523219,10.096846,1.375297,4.496868,1
1,0.245344,1.873901,2.177668,-2.907616,0.539491,3.113836,1.327698,10.024680,1.394138,5.226368,1
2,0.515188,2.052923,2.021482,-2.868717,-0.211045,3.082855,1.517711,10.090917,1.316772,4.904708,1
3,0.529766,1.793446,2.211673,-2.912831,0.281599,3.107664,1.510040,10.024062,1.379927,5.429069,1
4,0.821210,1.472261,2.057228,-2.733129,-0.566706,3.071136,1.462961,10.080511,1.363613,4.788601,1
5,0.006832,1.636809,2.006419,-3.058976,-0.320725,3.001691,1.342740,10.003274,1.012493,4.030539,1
6,0.210622,1.673330,2.118072,-2.981704,-0.725106,3.026343,1.441529,10.000217,1.149998,3.953387,1
7,-0.926146,1.261560,2.006528,-2.930952,0.484750,2.796875,1.414336,10.051943,1.113251,3.769814,1
8,-0.519451,1.875428,2.189321,-2.811001,-0.945430,2.944155,1.465402,10.063231,1.326599,2.944676,1
9,0.154951,2.140579,2.288953,-3.041501,-0.305526,3.018264,1.332776,10.012241,1.211707,4.200465,1


Con la variable x4 y x0 obtenemos el 100% de rendimiento en el modelo.

# Pregunta 2

Constuya un sistema clasificador que permita asignar un nuevo punto a una de las tres clases. El clasificador debe usar internamente la regresión logística. Es posible detectar las variables relevantes independientes usando regresión  logística? 

Para utilizar la Regresión Logística se utilizan 3 clasificadores, debido a que la variable dependiente tiene 3 clases. Dado que ninguna de las variables independientes son categóricas, estas se trabajan como decimales. Se normalizan las variables para que el modelo ajuste correctamente.

In [37]:
#Carga de las librerias necesarias
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

#Creación de las tres nuevas columnas las cuales son las variables dependientes de los 3 modelos (3 clasificadores  a partir de la regresión logistica)
df.insert(df.shape[1], "C1", ([0] * len(df)))
df.loc[df.iloc[:,(df.shape[1]-2)]==1,"C1"]=[1]

df.insert(df.shape[1], "C2", ([0] * len(df)))
df.loc[df.iloc[:,(df.shape[1]-3)]==2,"C2"]=[1]

df.insert(df.shape[1], "C3", ([0] * len(df)))
df.loc[df.iloc[:,(df.shape[1]-4)]==3,"C3"]=[1]

#Selección de variables independientes y dependientes de los 3 modelos (referentes a las 3 clases de la variable respueta)
indep=df.iloc[:,range(10)]
depend1=np.array(df.loc[:,"C1"], dtype=str)
depend2=np.array(df.loc[:,"C2"], dtype=str)
depend3=np.array(df.loc[:,"C3"], dtype=str)

#Normalizacion de variables
indep = (indep - indep.mean()) / (indep.max() - indep.min())

In [38]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,class,C1,C2,C3
0,0.217051,2.301448,2.063373,-2.877469,-0.280745,3.037343,1.523219,10.096846,1.375297,4.496868,1,1,0,0
1,0.245344,1.873901,2.177668,-2.907616,0.539491,3.113836,1.327698,10.024680,1.394138,5.226368,1,1,0,0
2,0.515188,2.052923,2.021482,-2.868717,-0.211045,3.082855,1.517711,10.090917,1.316772,4.904708,1,1,0,0
3,0.529766,1.793446,2.211673,-2.912831,0.281599,3.107664,1.510040,10.024062,1.379927,5.429069,1,1,0,0
4,0.821210,1.472261,2.057228,-2.733129,-0.566706,3.071136,1.462961,10.080511,1.363613,4.788601,1,1,0,0
5,0.006832,1.636809,2.006419,-3.058976,-0.320725,3.001691,1.342740,10.003274,1.012493,4.030539,1,1,0,0
6,0.210622,1.673330,2.118072,-2.981704,-0.725106,3.026343,1.441529,10.000217,1.149998,3.953387,1,1,0,0
7,-0.926146,1.261560,2.006528,-2.930952,0.484750,2.796875,1.414336,10.051943,1.113251,3.769814,1,1,0,0
8,-0.519451,1.875428,2.189321,-2.811001,-0.945430,2.944155,1.465402,10.063231,1.326599,2.944676,1,1,0,0
9,0.154951,2.140579,2.288953,-3.041501,-0.305526,3.018264,1.332776,10.012241,1.211707,4.200465,1,1,0,0


In [40]:
#Ajuste de cada uno de los modelos
model1 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl1 = model1.fit(indep, depend1)

model2 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl2 = model2.fit(indep, depend2)

model3 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl3 = model3.fit(indep, depend3)


#Predicción de cada uno de los modelos
predicted_1 = model1.predict(indep)
predicted_2 = model2.predict(indep)
predicted_3 = model3.predict(indep)


#Rendimiento de cada uno de los modelos
rend1=np.mean(depend1==predicted_1)
rend2=np.mean(depend2==predicted_2)
rend3=np.mean(depend3==predicted_3)

depend1=np.array(depend1, dtype=float)
depend2=np.array(depend2, dtype=float)
depend3=np.array(depend3, dtype=float)

#Rendimiento del modelo global
print("Rendimiento del modelo global:",(depend1.sum()*rend1+depend2.sum()*rend2+depend3.sum()*rend3)/indep.shape[0])

predicted_2

Rendimiento del modelo global: 1.0


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

Ahora se seleccionan las variables más importantes de cada uno de los modelos mediante la función SelectFromModel. Esto, debido a que cada uno de los clasificadores puede seleccionar variables diferentes.

In [5]:
model1_sf = SelectFromModel(model1, prefit=True)
indep1_new = model1_sf.transform(indep)

model2_sf = SelectFromModel(model2, prefit=True)
indep2_new = model2_sf.transform(indep)

model3_sf = SelectFromModel(model3, prefit=True)
indep3_new = model3_sf.transform(indep)

print("Variables independientes iniciales:   ",indep.shape[1],
      "\nVariables independientes seleccionadas:   ",indep1_new.shape[1])
print("Variables independientes iniciales:   ",indep.shape[1],
      "\nVariables independientes seleccionadas:   ",indep2_new.shape[1])
print("Variables independientes iniciales:   ",indep.shape[1],
      "\nVariables independientes seleccionadas:   ",indep3_new.shape[1])

print(indep1_new,
indep2_new,
indep3_new)

Variables independientes iniciales:    10 
Variables independientes seleccionadas:    3
Variables independientes iniciales:    10 
Variables independientes seleccionadas:    4
Variables independientes iniciales:    10 
Variables independientes seleccionadas:    4
[[-0.1392571  -0.23496336 -0.28978407]
 [-0.13449675 -0.11559366 -0.20473017]
 [-0.08909405 -0.22481985 -0.24223321]
 ..., 
 [-0.09864033  0.39669125  0.24812936]
 [-0.00932399  0.31928561  0.23812691]
 [-0.11849727  0.35555569  0.18120497]] [[-0.1392571  -0.23496336 -0.13876086 -0.28978407]
 [-0.13449675 -0.11559366 -0.07656122 -0.20473017]
 [-0.08909405 -0.22481985 -0.10175362 -0.24223321]
 ..., 
 [-0.09864033  0.39669125 -0.09012874  0.24812936]
 [-0.00932399  0.31928561 -0.08967742  0.23812691]
 [-0.11849727  0.35555569 -0.05566866  0.18120497]] [[-0.1392571  -0.23496336 -0.13876086 -0.28978407]
 [-0.13449675 -0.11559366 -0.07656122 -0.20473017]
 [-0.08909405 -0.22481985 -0.10175362 -0.24223321]
 ..., 
 [-0.09864033  0.396

De los 3 clasificadores, 1 de ellos seleccionó 2 variables, mientras que los otros 2 seleccionaron estas mismas 3 y un adicional. Así, puede decirse que el clasificador global considera 4 variables importantes:

In [6]:
indep3_new

array([[-0.1392571 , -0.23496336, -0.13876086, -0.28978407],
       [-0.13449675, -0.11559366, -0.07656122, -0.20473017],
       [-0.08909405, -0.22481985, -0.10175362, -0.24223321],
       ..., 
       [-0.09864033,  0.39669125, -0.09012874,  0.24812936],
       [-0.00932399,  0.31928561, -0.08967742,  0.23812691],
       [-0.11849727,  0.35555569, -0.05566866,  0.18120497]])

In [7]:
indep

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,-0.139257,0.143554,-0.282084,0.205907,-0.234963,-0.138761,0.305083,0.456962,0.418933,-0.289784
1,-0.134497,-0.093457,0.100915,0.157693,-0.115594,-0.076561,-0.163231,-0.269588,0.466112,-0.204730
2,-0.089094,0.005784,-0.422460,0.219904,-0.224820,-0.101754,0.291890,0.397263,0.272387,-0.242233
3,-0.086641,-0.138058,0.214865,0.149352,-0.153125,-0.081580,0.273516,-0.275817,0.430526,-0.181097
4,-0.037604,-0.316107,-0.302678,0.436753,-0.276579,-0.111283,0.160753,0.292506,0.389677,-0.255770
5,-0.174628,-0.224890,-0.472938,-0.084380,-0.240782,-0.167751,-0.127202,-0.485103,-0.489529,-0.344154
6,-0.140339,-0.204644,-0.098790,0.039202,-0.299632,-0.147706,0.109419,-0.515877,-0.145215,-0.353150
7,-0.331606,-0.432909,-0.472572,0.120372,-0.123560,-0.334296,0.044286,0.004887,-0.237231,-0.374553
8,-0.263177,-0.092610,0.139966,0.312211,-0.331696,-0.214536,0.166601,0.118531,0.296993,-0.470757
9,-0.149706,0.054376,0.473831,-0.056431,-0.238570,-0.154275,-0.151067,-0.394828,0.009305,-0.324342


Comparando las dos tablas, para el clasificador son importantes las variables  x1, x5, x6 y x10.

La predicción del modelo con las variables seleccionadas es la siguiente:

In [8]:
depend1=np.array(df.loc[:,"C1"], dtype=str)
depend2=np.array(df.loc[:,"C2"], dtype=str)
depend3=np.array(df.loc[:,"C3"], dtype=str)


#Ajuste de cada uno de los modelos
model1 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl1 = model1.fit(indep3_new, depend1)

model2 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl2 = model2.fit(indep3_new, depend2)

model3 = LogisticRegression(fit_intercept = True, C = 1e100)
mdl3 = model3.fit(indep3_new, depend3)


predicted_1 = model1.predict(indep3_new)
predicted_2 = model2.predict(indep3_new)
predicted_3 = model3.predict(indep3_new)


rend1=np.mean(depend1==predicted_1)
rend2=np.mean(depend2==predicted_2)
rend3=np.mean(depend3==predicted_3)

depend1=np.array(depend1, dtype=float)
depend2=np.array(depend2, dtype=float)
depend3=np.array(depend3, dtype=float)

#Rendimiento de cada uno de los modelos
print("Rendimiento de cada uno de los modelos:",rend1,rend2,rend3)

#Rendimiento del modelo global
print("Rendimiento del modelo global:",(depend1.sum()*rend1+depend2.sum()*rend2+depend3.sum()*rend3)/indep3_new.shape[0])

Rendimiento de cada uno de los modelos: 1.0 1.0 1.0
Rendimiento del modelo global: 1.0


# Pregunta 3

Construya un sistema clasificador usando la técnica de vecinos más próximos. Cómo podría modificar el algoritmo para detectar las variables más relevantes?

Se ajusta el modelo:

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel

#Selección del directorio de trabajo
os.chdir('/home/agch2/Dropbox/Asignaturas 2017-I MIS/Aprendizaje de maquinas/machine-learning-master')

#Carga del archivo en un dataframe
df = pd.read_table("datos.csv",sep=",")

#Selección de variables independientes y dependiente (la última no es usada para calibrar el modelo sino solo para ver que tal pronostica, ya que es un algoritmo no supervisado)
indep=df.iloc[:,range(10)]
depend=np.array(df.loc[:,"class"], dtype=int)

depend = np.array(df.loc[:,"class"], dtype=str)
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(indep, depend)

neigh_predict=neigh.predict(indep)

#Rendimiento del modelo:
rend_neigh=np.mean(depend==neigh_predict)
rend_neigh

1.0

Con vecinos más cercanos también se obtiene un rendimiento de 100%:

Se puede observar que la predicción del modelo es de un 100%. Ahora procedemos a seleccionar las variables principales. Al igual que K-means, la función de Python de vecinos más cercanos no cuenta con una función para la selección de variables. Así, se realiza la selección de variables de la siguiente forma:

- Se aplica el algoritmo añadiendo una por una cada variable independiente. Es decir, se empieza utilizando solamente una de ellas y luego se utiliza otra, extrayendo la variable incluida anteriormente. Esto se hace sucesivamente para cada una de las variables. De todas las variables, finalmente la primera variable que se incluye en el modelo, es quella que mayor aumento genera en su rendimiento.
- El procedimiento anterior se hace repetidamente hasta que el rendimiento se estabilice.

In [10]:
rend_tot_fin=[]
S_F_tot=[]
V_S_F=range(indep.shape[1])
indep2=indep[:]


for j in range(indep.shape[1]):  #Ver como cambia el rendimiento mientras aumento una por una mis variables

    rend_tot=[]
    
    for i in V_S_F:  #Selecionar la variable que genera mayor aumento en el rendimiento de cada corrida
        
        if j==0:
            indep_F_S=indep.iloc[:,[i]]
        else:
            indep_F_S=indep.iloc[:,list(set().union(S_F_tot,[i]))]
            
        #print(list(set().union(seq1,seq2)))
        #print(indep_F_S)
        
        
        
        
        neigh = KNeighborsClassifier(n_neighbors=3)

        neigh.fit(indep_F_S, depend)

        neigh_predict=neigh.predict(indep_F_S)

        #Rendimiento del modelo:
        rend_neigh=np.mean(depend==neigh_predict)
        rend_neigh
        

        #rend_Km
        #print("%.3f" % rend_Km)

        rend_tot.append(rend_neigh)

    #print(rend_tot)
    m = max(rend_tot)
    S_F=[i for i, j in enumerate(rend_tot) if j == m]
    if j==0:
        S_F=S_F[:]
    else:
        for k in S_F_tot:
            if S_F[0]>=k:
                S_F[0]=S_F[0]+1
            else: S_F[0]=S_F[0]
    S_F_tot.append(S_F[0])
    
    seq1=list(range(indep2.shape[1]))[0:S_F[0]]
    seq2=list(range(indep2.shape[1]))[S_F[0]+1:10]
    V_S_F=list(set().union(seq1,seq2))
    indep2=indep2.iloc[:,V_S_F]
    #print(V_S_F)
    
    rend_tot_fin.append(m)

#print(rend_tot_fin)

m2 = max(rend_tot_fin)
S_F_Max=[i for i, j in enumerate(rend_tot_fin) if j == m2]

S_F_Variables=S_F_tot[0:S_F_Max[0]+1]
print("Variables seleccionadas:   ","x",S_F_Variables[0],",   x",S_F_Variables[1],"\nRendimiento:",m2)

Variables seleccionadas:    x 4 ,   x 0 
Rendimiento: 1.0


# Pregunta 4

Construya un sistema clasificador usando árboles de decisión. Ya que los árboles permiten detectar variables relevantes, las variables seleccionadas en este punto coinciden con las obtenidas en las preguntas anteriores?

In [11]:
from sklearn import tree
from sklearn.feature_selection import SelectFromModel

depend = np.array(df.loc[:,"class"], dtype=str)
treeC = tree.DecisionTreeClassifier()
treeC = treeC.fit(indep, depend)

predicted=treeC.predict(indep)
rend=np.mean(depend==predicted)

print(rend)
#print(treeC.predict(indep))
#print(treeC.predict_proba(indep))

1.0


Puede observarse que la predicción del modelo es de 100%. Ahora, se observan las variables más importantes:

In [12]:
treeC.feature_importances_

array([ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0.5])

Genero el modelo con las variables más importantes:

In [13]:
treeC_sf = SelectFromModel(treeC, prefit=True)
indep_treeC_new = treeC_sf.transform(indep)
indep_treeC_new

array([[ -2.80745077e-01,   4.49686781e+00],
       [  5.39491346e-01,   5.22636841e+00],
       [ -2.11045031e-01,   4.90470781e+00],
       [  2.81598929e-01,   5.42906859e+00],
       [ -5.66705821e-01,   4.78860103e+00],
       [ -3.20724736e-01,   4.03053851e+00],
       [ -7.25106368e-01,   3.95338673e+00],
       [  4.84749562e-01,   3.76981427e+00],
       [ -9.45430259e-01,   2.94467622e+00],
       [ -3.05525688e-01,   4.20046519e+00],
       [ -3.74175287e-01,   3.58469583e+00],
       [ -6.60614728e-01,   3.37249152e+00],
       [  3.96413730e-01,   4.15868567e+00],
       [ -7.20570106e-01,   3.60498869e+00],
       [ -4.37384788e-01,   2.94222910e+00],
       [ -3.90904119e-01,   3.76264235e+00],
       [  3.06888599e-01,   5.78029504e+00],
       [ -1.09621672e+00,   3.45927769e+00],
       [ -5.31482263e-01,   4.45017124e+00],
       [ -5.53718518e-01,   3.66071087e+00],
       [  3.92647128e-01,   4.64097982e+00],
       [  6.11643474e-01,   4.98200776e+00],
       [ -

In [14]:
indep

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,0.217051,2.301448,2.063373,-2.877469,-0.280745,3.037343,1.523219,10.096846,1.375297,4.496868
1,0.245344,1.873901,2.177668,-2.907616,0.539491,3.113836,1.327698,10.024680,1.394138,5.226368
2,0.515188,2.052923,2.021482,-2.868717,-0.211045,3.082855,1.517711,10.090917,1.316772,4.904708
3,0.529766,1.793446,2.211673,-2.912831,0.281599,3.107664,1.510040,10.024062,1.379927,5.429069
4,0.821210,1.472261,2.057228,-2.733129,-0.566706,3.071136,1.462961,10.080511,1.363613,4.788601
5,0.006832,1.636809,2.006419,-3.058976,-0.320725,3.001691,1.342740,10.003274,1.012493,4.030539
6,0.210622,1.673330,2.118072,-2.981704,-0.725106,3.026343,1.441529,10.000217,1.149998,3.953387
7,-0.926146,1.261560,2.006528,-2.930952,0.484750,2.796875,1.414336,10.051943,1.113251,3.769814
8,-0.519451,1.875428,2.189321,-2.811001,-0.945430,2.944155,1.465402,10.063231,1.326599,2.944676
9,0.154951,2.140579,2.288953,-3.041501,-0.305526,3.018264,1.332776,10.012241,1.211707,4.200465


Las variables seleccionadas son x6 y x10, lo cual difiere con los anteriores modelos.

El rendimiento del modelo con las variables seleccionadas es de 100%:

In [15]:
treeC = tree.DecisionTreeClassifier()
treeC = treeC.fit(indep_treeC_new, depend)

predicted = treeC.predict(indep_treeC_new)

rend=np.mean(depend==predicted)
rend

1.0

---